**This is the simplified Opensantions data in csv format**

In [1]:
#import packages
import requests
import pandas as pd
import numpy as np
import os
import re

# OFAC SDN

In [2]:
#I can access the data via pandas very easily, output already in tabular form
data = pd.read_csv('https://data.opensanctions.org/datasets/20240121/us_ofac_sdn/targets.simple.csv')
data

,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,first_seen,last_seen,last_change
0,NK-22HtK7WrxZ2sU3rmhz6PuZ,Person,Michael Kuajien,Michael Kuajian;Michael Kuajien Duer Mayok,1979-01-01,ke;ss,Nairobi,NaN,SDN List - Executive Order 13818 (Global Magni...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2023-04-20T10:27:20,2024-01-21T22:10:01,2023-11-17T08:23:17
1,NK-22oMG6jqPQknWaMjzTn4hK,Organization,Limited Liability Company Garantiya,Garantiya OOO,NaN,ru,"bulvar Tverskoi, d. 15 str. 2, Moscow",5067746901426;7703610362,SDN List - Executive Order 14024 (Russia),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2023-04-20T10:27:20,2024-01-21T22:10:01,2023-11-17T08:23:17
2,NK-23p2d4vMT5sJtQ845GyzJt,Organization,Scientific and Production Association of Measu...,AO NPO IT;Aktsionernoe Obschestvo Nauchno Proi...,NaN,ru,"2k4 Pionerskaya Str., 141074 Korolyov",1095018006555;5018139517,SDN List - Executive Order 14024 (Russia),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2023-04-20T10:27:20,2024-01-21T22:10:01,2023-11-17T08:23:17
3,NK-23rgYEXa9AHtupZKgS8Tbc,Person,Bahram Ali SHAYESTEH,Bahrami Ali JADALI;Bahrami Ali SHAYESTEH,1958-06-13;1963-05-06;1963-08-06,de,"80331 Muenchen, Bayern",NaN,SDN List - Unknown,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2023-04-20T10:27:20,2024-01-21T22:10:01,2023-11-17T08:23:17
4,NK-24UMVkuxF3svsAtg55i3Zi,Organization,ALBAHR ALAAHMAR ENERGY FZE,البحر الأحمر للطاقة م م ح,NaN,ae,"P1-ELOB Office No. E-44G-29, Hamriyah Free Zon...",11670819;19438,SDN List - Executive Order 13846 (Iran),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2023-04-20T10:27:20,2024-01-21T22:10:01,2023-11-17T08:23:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13799,ofac-47225,Vessel,Peria,NaN,NaN,lr,NaN,5LIZ6;636022479;IMO 9322827,SDN List - Executive Order 14024 (Russia),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2024-01-18T16:10:01,2024-01-21T22:10:01,2024-01-18T16:10:01
13800,ofac-47226,Vessel,Sara II,NaN,NaN,lr,NaN,5LJI4;636022546;IMO 9301615,SDN List - Executive Order 14024 (Russia),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2024-01-18T16:10:01,2024-01-21T22:10:01,2024-01-18T16:10:01
13801,ofac-47227,Vessel,Sensus,NaN,NaN,lr,NaN,5LHJ7;636022146;IMO 9296585,SDN List - Executive Order 14024 (Russia),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2024-01-18T16:10:01,2024-01-21T22:10:01,2024-01-18T16:10:01
13802,ofac-47228,Vessel,Uze,NaN,NaN,lr,NaN,5LHB3;636022072;IMO 9323338,SDN List - Executive Order 14024 (Russia),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2024-01-18T16:10:01,2024-01-21T22:10:01,2024-01-18T16:10:01


In [3]:
#create urls for all relevant dates
date_list = pd.date_range(start='20220117',end='20220301',freq='D').strftime('%Y%m%d')
date_list

Index(['20220117', '20220118', '20220119', '20220120', '20220121', '20220122',
       '20220123', '20220124', '20220125', '20220126', '20220127', '20220128',
       '20220129', '20220130', '20220131', '20220201', '20220202', '20220203',
       '20220204', '20220205', '20220206', '20220207', '20220208', '20220209',
       '20220210', '20220211', '20220212', '20220213', '20220214', '20220215',
       '20220216', '20220217', '20220218', '20220219', '20220220', '20220221',
       '20220222', '20220223', '20220224', '20220225', '20220226', '20220227',
       '20220228', '20220301'],
      dtype='object')

In [4]:
#get all the urls for all the dates we need the data for
#make each day an individuallist so we can compare and match them
websites = []

for i in date_list:
    test = 'https://data.opensanctions.org/datasets/'+(i)+'/us_ofac_sdn/targets.simple.csv'
    websites.append(test)
print(websites)

['https://data.opensanctions.org/datasets/20220117/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220118/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220119/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220120/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220121/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220122/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220123/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220124/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220125/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220126/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220127/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220128/us_ofac_sdn/targets.simple.csv', 'ht

In [5]:
#add this to loop to identify missing dates:
#for site in websites:
       # try:
            #response = requests.get(site)
            #response.raise_for_status()  # Raises an HTTPError for bad response
        #except requests.exceptions.RequestException as e:
                #print(f"Error accessing {site}: {e}")
                #continue

In [6]:
#now loop over the batches to get all the data and append the dataframes to one another
#this gives us a list of dataframes
#the loop must contain a date identifyer as a new column so each dataframe is marked with the retrieval date that is then needed to match the different days against each other
entities_list = []
date_pattern = r'/datasets/(\d{8})/'
for site in websites:
    data = pd.read_csv(site)
    match = re.search(date_pattern, site) #extract the date from the url
    if match:
        date = match.group(1)
        data['date_stamp'] = date #set an individual date stamp for each dataframe matching the url date
        #ata =  data.set_index('date_stamp') #set as an index
    entities_list.append(data)
res = pd.concat(entities_list)  # concatenate list of dataframes

In [7]:
res

,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,last_seen,first_seen,date_stamp
0,NK-22HtK7WrxZ2sU3rmhz6PuZ,Person,Michael Kuajien,Michael Kuajian;Michael Kuajien Duer Mayok,1979-01-01,ke;ss,Nairobi;South Sudan,NaN,SDN List - Program - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-17 07:40:14,20220117
1,NK-23rgYEXa9AHtupZKgS8Tbc,Person,Bahram Ali SHAYESTEH,Bahrami Ali JADALI;Bahrami Ali SHAYESTEH,1958-06-13;1963-05-06;1963-08-06,de,"80331 Muenchen, Bayern",NaN,SDN List - Program - Block - Unknown - 2011-04-28,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-17 07:40:14,20220117
2,NK-24cEBmXGjPPVJRwMPGx3Yj,Company,"LOLA LOLITA 1110, S. DE R.L. DE C.V.","LOLALOLITA 1110, S. DE R.L. DE C.V.",NaN,mx,"Av. Vallarta 1110, Col. Americana, 44160 Guada...",60645 (Jalisco),SDN List - Program - Block - Foreign Narcotics...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-17 07:40:14,20220117
3,NK-24KmksG96rQedGYXzm4xHU,Organization,CONSTRUCCIONES E INVERSIONES LTDA.,NaN,NaN,co,"Calle 15 No. 10-52, La Union, Valle",800154939-3,SDN List - Program - Block - Unknown - 2006-11-28,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-17 07:40:14,20220117
4,NK-24kqBFxEsFzjqBYhaNHkC9,Organization,TAWU Mechanical Engineering and Trading Company,Metaalkunde BV;TAWU;TAWU BVBA;TAWU BVBA Mechan...,NaN,be,"Bleidenhoek 34, 2230 Herselt",686896689;BE0686.896.689;BE0686896689,SDN List - Program - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-17 07:40:14,20220117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9201,Q983654,Person,Sergey Yevgenyevich NARYSHKIN,Sergei NARYSHKIN,1954-10-27,NaN,NaN,NaN,SDN List - Block - Program - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-03-01 18:47:01,20220301
9202,Q983894,Person,Vujadin POPOVIC,NaN,1957-03-14,NaN,NaN,NaN,SDN List - Block - Program - Unknown,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-03-01 18:47:01,20220301
9203,Q984195,Person,Dragomir MILOSEVIC,NaN,1942-02-04,NaN,NaN,NaN,SDN List - Block - Program - Unknown,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-03-01 18:47:01,20220301
9204,Q98831330,Person,Ivan Uladzimiravich KUBRAKOU,Ivan Kubrakov;Ivan Vladimirovich Kubrakov;Іван...,1975-05-05,by,Belarus;Minsk,NaN,SDN List - Block - Program - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-03-01 18:47:01,20220301


In [8]:
#now match the different dates against each other and return an indicator per row for a new listing or delisting
# Sort the DataFrame based on date_stamp and id
res.sort_values(by=['id', 'date_stamp'], inplace=True)

# Identify new entries and deletions based on the index and columns you want to compare
new_entries = ~res.duplicated(subset=['id'], keep='first')
deletions = ~res.duplicated(subset=['id'], keep='last')

# Identify unchanged rows by checking for duplicates based on the id and date_stamp
unchanged_rows = ~new_entries & ~deletions

# Set the values of new_entry, deletion, and unchanged columns
res['new_entry'] = new_entries
res['deletion'] = deletions
res['unchanged'] = unchanged_rows

# Reset index for the final result
res.reset_index(drop=True, inplace=True)

In [9]:
res

,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,last_seen,first_seen,date_stamp,new_entry,deletion,unchanged
0,NK-22HtK7WrxZ2sU3rmhz6PuZ,Person,Michael Kuajien,Michael Kuajian;Michael Kuajien Duer Mayok,1979-01-01,ke;ss,Nairobi;South Sudan,NaN,SDN List - Program - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-17 07:40:14,20220117,True,False,False
1,NK-22HtK7WrxZ2sU3rmhz6PuZ,Person,Michael Kuajien,Michael Kuajian;Michael Kuajien Duer Mayok,1979-01-01,ke;ss,Nairobi;South Sudan,NaN,Program - SDN List - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-18 01:30:55,20220118,False,False,True
2,NK-22HtK7WrxZ2sU3rmhz6PuZ,Person,Michael Kuajien,Michael Kuajian;Michael Kuajien Duer Mayok,1979-01-01,ke;ss,"Nairobi, 248-00100;South Sudan",NaN,SDN List - Program - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-19 01:27:13,20220119,False,False,True
3,NK-22HtK7WrxZ2sU3rmhz6PuZ,Person,Michael Kuajien,Michael Kuajian;Michael Kuajien Duer Mayok,1979-01-01,ke;ss,"Nairobi, 248-00100;South Sudan",NaN,Program - Block - SDN List - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-20 01:33:01,20220120,False,False,True
4,NK-22HtK7WrxZ2sU3rmhz6PuZ,Person,Michael Kuajien,Michael Kuajian;Michael Kuajien Duer Mayok,1979-01-01,ke;ss,"Nairobi, 248-00100;South Sudan",NaN,SDN List - Program - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-21 01:28:08,20220121,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398504,ofac-35173,Person,Sergei Shoigu,NaN,1955-05-21,NaN,NaN,NaN,Program - SDN List - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2022-02-26 01:06:48,2022-02-26 18:23:44,20220226,True,True,False
398505,ofac-35176,Person,Valery Gerasimov,NaN,1955-09-08,NaN,NaN,NaN,Program - SDN List - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2022-02-26 01:06:48,2022-02-26 18:23:44,20220226,True,True,False
398506,ofac-35233,Person,Sergei Viktorovich Lavrov,Sergey Lavrov;Сергей Викторович Лавров,1950-03-21,NaN,NaN,NaN,Program - SDN List - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2022-02-26 01:06:48,2022-02-26 18:23:44,20220226,True,True,False
398507,ofac-35244,Company,LIMITED LIABILITY COMPANY RVC MANAGEMENT COMPANY,LLC MC RVC;LLC UK RVK;OOO UK RVK;Obshchestvo s...,NaN,ru,"D. 8, Str. 1, Etaj 12, Nab. Presnenskaya, Mosc...",1207700502547;33185693;9703024347,Program - SDN List - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2022-02-28 18:27:19,2022-02-28 18:27:19,20220228,True,False,False


In [10]:
#test thematching function
test = res.query('new_entry == True')
test

,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,last_seen,first_seen,date_stamp,new_entry,deletion,unchanged
0,NK-22HtK7WrxZ2sU3rmhz6PuZ,Person,Michael Kuajien,Michael Kuajian;Michael Kuajien Duer Mayok,1979-01-01,ke;ss,Nairobi;South Sudan,NaN,SDN List - Program - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-17 07:40:14,20220117,True,False,False
44,NK-23rgYEXa9AHtupZKgS8Tbc,Person,Bahram Ali SHAYESTEH,Bahrami Ali JADALI;Bahrami Ali SHAYESTEH,1958-06-13;1963-05-06;1963-08-06,de,"80331 Muenchen, Bayern",NaN,SDN List - Program - Block - Unknown - 2011-04-28,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-17 07:40:14,20220117,True,False,False
88,NK-24KmksG96rQedGYXzm4xHU,Organization,CONSTRUCCIONES E INVERSIONES LTDA.,NaN,NaN,co,"Calle 15 No. 10-52, La Union, Valle",800154939-3,SDN List - Program - Block - Unknown - 2006-11-28,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-17 07:40:14,20220117,True,False,False
132,NK-24cEBmXGjPPVJRwMPGx3Yj,Company,"LOLA LOLITA 1110, S. DE R.L. DE C.V.","LOLALOLITA 1110, S. DE R.L. DE C.V.",NaN,mx,"Av. Vallarta 1110, Col. Americana, 44160 Guada...",60645 (Jalisco),SDN List - Program - Block - Foreign Narcotics...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-17 07:40:14,20220117,True,False,False
176,NK-24kqBFxEsFzjqBYhaNHkC9,Organization,TAWU Mechanical Engineering and Trading Company,Metaalkunde BV;TAWU;TAWU BVBA;TAWU BVBA Mechan...,NaN,be,"Bleidenhoek 34, 2230 Herselt",686896689;BE0686.896.689;BE0686896689,SDN List - Program - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-17 07:40:14,20220117,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398503,ofac-35096,Person,Vladimir Vladimirovich Putin,Vladimir Putin;Владимир Владимирович Путин;Вла...,1952-10-07,ru,"Bocharov Ruchey, Sochi;Kremlin, Moscow;Novo-Og...",NaN,Program - SDN List - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2022-02-26 01:06:48,2022-02-26 18:23:44,20220226,True,True,False
398504,ofac-35173,Person,Sergei Shoigu,NaN,1955-05-21,NaN,NaN,NaN,Program - SDN List - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2022-02-26 01:06:48,2022-02-26 18:23:44,20220226,True,True,False
398505,ofac-35176,Person,Valery Gerasimov,NaN,1955-09-08,NaN,NaN,NaN,Program - SDN List - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2022-02-26 01:06:48,2022-02-26 18:23:44,20220226,True,True,False
398506,ofac-35233,Person,Sergei Viktorovich Lavrov,Sergey Lavrov;Сергей Викторович Лавров,1950-03-21,NaN,NaN,NaN,Program - SDN List - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2022-02-26 01:06:48,2022-02-26 18:23:44,20220226,True,True,False


In [11]:
res_deduplicated = res.copy()
res_deduplicated.sort_values(by=['id', 'date_stamp'], inplace=True)

# Identify new entries and deletions based on the index and columns you want to compare
new_entries = ~res_deduplicated.duplicated(subset=['id'], keep='first')
deletions = ~res_deduplicated.duplicated(subset=['id'], keep='last')

# Identify unchanged rows by checking for duplicates based on the id and date_stamp
unchanged_rows = ~new_entries & ~deletions

# Create new columns for first seen and last seen dates
res_deduplicated['listing_date'] = res_deduplicated['date_stamp'].where(new_entries)
res_deduplicated['delisting_date'] = res_deduplicated['date_stamp'].where(deletions)

# Add a 'month' column
res_deduplicated['month'] = pd.to_datetime(res_deduplicated['date_stamp']).dt.to_period('M')

# Deduplicate the entries (keep the first occurrence for each entity)
res_deduplicated = res_deduplicated.drop_duplicates(subset=['id'], keep='first')

# Reset index for the final result
res_deduplicated.reset_index(drop=True, inplace=True)

# Print or further analyze the deduplicated DataFrame 'res_deduplicated'
print(res_deduplicated)

                             id        schema  \
0     NK-22HtK7WrxZ2sU3rmhz6PuZ        Person   
1     NK-23rgYEXa9AHtupZKgS8Tbc        Person   
2     NK-24KmksG96rQedGYXzm4xHU  Organization   
3     NK-24cEBmXGjPPVJRwMPGx3Yj       Company   
4     NK-24kqBFxEsFzjqBYhaNHkC9  Organization   
...                         ...           ...   
9577                 ofac-35096        Person   
9578                 ofac-35173        Person   
9579                 ofac-35176        Person   
9580                 ofac-35233        Person   
9581                 ofac-35244       Company   

                                                  name  \
0                                      Michael Kuajien   
1                                 Bahram Ali SHAYESTEH   
2                   CONSTRUCCIONES E INVERSIONES LTDA.   
3                 LOLA LOLITA 1110, S. DE R.L. DE C.V.   
4      TAWU Mechanical Engineering and Trading Company   
...                                                ...   
9577 

In [12]:
res_deduplicated

,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,...,dataset,last_seen,first_seen,date_stamp,new_entry,deletion,unchanged,listing_date,delisting_date,month
0,NK-22HtK7WrxZ2sU3rmhz6PuZ,Person,Michael Kuajien,Michael Kuajian;Michael Kuajien Duer Mayok,1979-01-01,ke;ss,Nairobi;South Sudan,NaN,SDN List - Program - Block - Executive Order 1...,NaN,...,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-17 07:40:14,20220117,True,False,False,20220117,NaN,2022-01
1,NK-23rgYEXa9AHtupZKgS8Tbc,Person,Bahram Ali SHAYESTEH,Bahrami Ali JADALI;Bahrami Ali SHAYESTEH,1958-06-13;1963-05-06;1963-08-06,de,"80331 Muenchen, Bayern",NaN,SDN List - Program - Block - Unknown - 2011-04-28,NaN,...,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-17 07:40:14,20220117,True,False,False,20220117,NaN,2022-01
2,NK-24KmksG96rQedGYXzm4xHU,Organization,CONSTRUCCIONES E INVERSIONES LTDA.,NaN,NaN,co,"Calle 15 No. 10-52, La Union, Valle",800154939-3,SDN List - Program - Block - Unknown - 2006-11-28,NaN,...,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-17 07:40:14,20220117,True,False,False,20220117,NaN,2022-01
3,NK-24cEBmXGjPPVJRwMPGx3Yj,Company,"LOLA LOLITA 1110, S. DE R.L. DE C.V.","LOLALOLITA 1110, S. DE R.L. DE C.V.",NaN,mx,"Av. Vallarta 1110, Col. Americana, 44160 Guada...",60645 (Jalisco),SDN List - Program - Block - Foreign Narcotics...,NaN,...,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-17 07:40:14,20220117,True,False,False,20220117,NaN,2022-01
4,NK-24kqBFxEsFzjqBYhaNHkC9,Organization,TAWU Mechanical Engineering and Trading Company,Metaalkunde BV;TAWU;TAWU BVBA;TAWU BVBA Mechan...,NaN,be,"Bleidenhoek 34, 2230 Herselt",686896689;BE0686.896.689;BE0686896689,SDN List - Program - Block - Executive Order 1...,NaN,...,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-17 07:40:14,20220117,True,False,False,20220117,NaN,2022-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9577,ofac-35096,Person,Vladimir Vladimirovich Putin,Vladimir Putin;Владимир Владимирович Путин;Вла...,1952-10-07,ru,"Bocharov Ruchey, Sochi;Kremlin, Moscow;Novo-Og...",NaN,Program - SDN List - Block - Executive Order 1...,NaN,...,US OFAC Specially Designated Nationals (SDN) List,2022-02-26 01:06:48,2022-02-26 18:23:44,20220226,True,True,False,20220226,20220226,2022-02
9578,ofac-35173,Person,Sergei Shoigu,NaN,1955-05-21,NaN,NaN,NaN,Program - SDN List - Block - Executive Order 1...,NaN,...,US OFAC Specially Designated Nationals (SDN) List,2022-02-26 01:06:48,2022-02-26 18:23:44,20220226,True,True,False,20220226,20220226,2022-02
9579,ofac-35176,Person,Valery Gerasimov,NaN,1955-09-08,NaN,NaN,NaN,Program - SDN List - Block - Executive Order 1...,NaN,...,US OFAC Specially Designated Nationals (SDN) List,2022-02-26 01:06:48,2022-02-26 18:23:44,20220226,True,True,False,20220226,20220226,2022-02
9580,ofac-35233,Person,Sergei Viktorovich Lavrov,Sergey Lavrov;Сергей Викторович Лавров,1950-03-21,NaN,NaN,NaN,Program - SDN List - Block - Executive Order 1...,NaN,...,US OFAC Specially Designated Nationals (SDN) List,2022-02-26 01:06:48,2022-02-26 18:23:44,20220226,True,True,False,20220226,20220226,2022-02


In [16]:
#the countries are coded as iso2 codes -> transform to iso 3
#keep both countries? could search for "ru" over regex functions
print(res_deduplicated.countries.unique())
print(res_deduplicated.schema.unique())
print(res_deduplicated.listing_date.unique())

['ke;ss' 'de' 'co' 'mx' 'be' 'il;md;ru' nan 'ru' 'iq' 'gb;ir' 'zw' 'pk'
 'ir' 'kp' 'cn' 'tw' 've' 'co;es;pa' 'cy' 'by' 'mt' 'ps' 'vc' 'mx;us' 'lb'
 'my' 'jp' 'us' 'km' 'sy' 'ae;ru;uz' 'eg' 'sa' 'pk;sa' 'sk;ve' 'au' 'mr'
 'ae' 'mm' 'gb' 'lb;ng;sl' 'sg' 'jo' 'ar' 'lv;tr' 'ca' 'rs' 'bg' 'it' 'sv'
 'at' 'pa' 'lv;ru' 'ni' 'ae;in' 'tr' 'co;ve'
 'ar;bg;ca;de;es;it;kp;nl;pl;ru' 'ae;gb;sd' 'iq;tr' 'er;et' 'cn;kp'
 'ae;sy' 'ht' 'it;tn' 'af;pk' 'ly' 'ye' 'pa;tr' 'ae;lr' 'za' 'de;jo'
 'ru;ua' 'es' 'de;iq' 'cn;hk' 'iq;ir' 'er;ke' 'tj' 'co;lb' 'ch;iq' 'cf'
 'ae;kg' 'vn' 'cn;vg' 'do' 'gt' 'vg' 'ph' 'il' 'cz;sc' 'kh' 'th'
 'ca;de;us' 'kg' 'in' 'gr' 'gb;jo'
 'ao;ar;bb;ca;cz;de;es;fr;gb;gy;ht;jm;mx;pa;pe;ru' 'ua'
 'cn;eg;hk;kp;pe;ru;th' 'mh;tr' 'co;pa' 'hn' 'af' 'tg' 'bd' 'tz' 'me;rs'
 'ch' 'ug' 'mm;th' 'bj' 'iq;sy' 'iq;no' 'co;mx' 'de;ma' 'ws' 'pe' 'me'
 'dz' 'bs' 'so' 'by;ru' 'om' 'lr' 'cn;hk;th' 'ch;ir' 'gb;jo;us' 'se'
 'ir;sy' 'hk' 'sd' 'iq;jo' 'ae;gb' 'ae;sc' 'cu;id' 'gr;ru' 'fr;lb' 'ss'
 'ky' 'bz;

In [25]:
list_regimes = list[res_deduplicated.sanctions.unique()]
print(list_regimes)
#but it doesnt show me all

list[array(['SDN List - Program - Block - Executive Order 13818 (Global Magnitsky) - 2019-12-10',
       'SDN List - Program - Block - Unknown - 2011-04-28',
       'SDN List - Program - Block - Unknown - 2006-11-28', ...,
       'Program - Block - SDN List - Executive Order 13224 (Terrorism) - 2022-02-23',
       'SDN List - Block - Program - Executive Order 13224 (Terrorism)',
       'Program - SDN List - Block - Executive Order 14024 - 2022-02-25'],
      dtype=object)]


In [26]:
#get missingness
is_null = res_deduplicated.isnull().sum()
display(is_null)

id                   0
schema               0
name                 0
aliases           4316
birth_date        4721
countries         1984
addresses         2797
identifiers       4480
sanctions            0
phones            9572
emails            9437
dataset              0
date_stamp           0
listing_date         0
delisting_date    9539
month                0
dtype: int64

In [19]:
res_deduplicated.columns

Index(['id', 'schema', 'name', 'aliases', 'birth_date', 'countries',
       'addresses', 'identifiers', 'sanctions', 'phones', 'emails', 'dataset',
       'date_stamp', 'listing_date', 'delisting_date', 'month'],
      dtype='object')

In [18]:
res_deduplicated = res_deduplicated.drop([ 'last_seen', 'first_seen','new_entry', 'deletion','unchanged'],axis = 1)

In [20]:
res_deduplicated.to_csv("ofac_sdn_test.csv")